In [1]:
from __future__ import print_function
#from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf
from tensorflow.python.ops import rnn_cell, rnn

In [12]:
from __future__ import division
import time

In [3]:
!ls '/home/htan/proj/TensorFlow/data/visibility/'

ASOS_alone.pickle	     CC_Under_sSample.pickle
ASOS+NWP.pickle		     KORD_RUC_RAP_Hourly_20051031-20150301.csv
ASOS+NWP_time_serial.pickle  rough_visibility.pickle
ASOS_time_serial.pickle      visibilityDataNoLagsNewPreds.csv


In [4]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS+NWP_time_serial.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    #train_dataset= save['train_dataset']
    #validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    #test_old = save['v_t_dataset']
    train_old = save['t_v_dataset']
    del save

#train_time = train_dataset['time']
#train_data = train_dataset['data']
#train_label = train_dataset['label']
#validate_time = validate_dataset['time']
#validate_data = validate_dataset['data']
#validate_label = validate_dataset['label']
#test_time = test_dataset['time']
test_data = test_dataset['data']
test_label = test_dataset['label']
#test_old_data = test_old['data']
#test_old_label = test_old['label']
train_old_data = train_old['data']
train_old_label = train_old['label']

In [5]:
print(test_data.shape, test_label.shape)
print(train_old_data.shape, train_old_label.shape)

(17664, 8, 82) (17664, 8, 1)
(70656, 8, 82) (70656, 8, 1)


In [6]:
#dataset normalize
mean = train_old_data.mean(axis = (0,1))
std = train_old_data.std(axis = (0,1))
print(mean.shape, std.shape)
train_data_n = (train_old_data - mean)/std
#validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std

(82,) (82,)


In [7]:
train_label = train_old_label

# Try LSTM + CNN 

In [17]:
batch_size = 128
patch_size = 7
n_hidden = 64
n_features1 = 70
n_features2 = 12
n_steps = 8
n_labels = 1


total_size = train_old_label.shape[0]
print(total_size)

70656


In [18]:


graph = tf.Graph()
with graph.as_default():
    inputs1 = tf.placeholder("float32", [None, n_steps, n_features1])
    inputs2 = tf.placeholder("float32", [None, n_steps, n_features2])
    labels = tf.placeholder("float32", [None, n_labels])

    weights = {
        'layer1': tf.Variable(tf.truncated_normal([patch_size, 1, 1, 16], stddev=0.1)),
        'layer2': tf.Variable(tf.truncated_normal([patch_size, 1, 16, 16], stddev=0.1)),
        'layer3': tf.Variable(tf.truncated_normal([10 * 7 * 16, n_hidden], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([n_hidden+12, n_labels])),
    }
    biases = {
        'layer1': tf.Variable(tf.zeros([16])),
        'layer2': tf.Variable(tf.constant(1.0, shape=[16])),
        'layer3': tf.Variable(tf.constant(1.0, shape=[n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels])),
    }
    
    def cov_model(data, w, b):
        conv = tf.nn.conv2d(data, w['layer1'], [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b['layer1'])
        conv = tf.nn.conv2d(hidden, w['layer2'], [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b['layer2'])
        #shape = hidden.get_shape().as_list()
        shape = tf.pack([tf.shape(hidden)[0], tf.shape(hidden)[1]*tf.shape(hidden)[2]*tf.shape(hidden)[3]])
        reshape = tf.reshape(hidden, shape)
        hidden = tf.nn.relu(tf.matmul(reshape, w['layer3']) + b['layer3'])
        return hidden
    
    def RNN(x, y, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        y = tf.transpose(y, [1, 0, 2])
        
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features1])
        y = tf.reshape(y, [-1, n_features2])
        # Linear activation
        x = tf.reshape(x, [-1, 7, 10, 1])
        x = cov_model(x, w, b)
        
        concat = tf.concat(1, [x, y])
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        list_data = tf.split(0, n_steps, concat)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden+12, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, list_data, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs1, inputs2, weights, biases)
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.abs(pred - labels))
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    MAE = tf.reduce_mean(tf.abs(pred - labels))

In [19]:
batch_size = 128
steps = 10000
st = time.time()
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs1:test_data_n[:,:,:70], inputs2:test_data_n[:,:,70:], labels:test_label[:,-1,:]}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n[off:off+batch_size, :, :]
        batch_label = train_label[off:off+batch_size, -1, :]
        feed_dict = {inputs1:batch_data[:,:,:70], inputs2:batch_data[:,:,70:], labels:batch_label}
        l, _, r = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 100 == 0:
            test_mae = MAE.eval(feed_dict=test_feed_dict)
            print('step: %d, LR = %f, min batch loss = %f, test MAE = %f' % (step, r, l, test_mae))
et = time.time()
print(et-st)

Initialized
step: 0, LR = 0.010000, min batch loss = 4.305766, test MAE = 10.312125
step: 100, LR = 0.010000, min batch loss = 0.278466, test MAE = 1.314774
step: 200, LR = 0.010000, min batch loss = 0.518329, test MAE = 1.330752
step: 300, LR = 0.010000, min batch loss = 0.782156, test MAE = 1.230568
step: 400, LR = 0.010000, min batch loss = 0.688073, test MAE = 1.223150
step: 500, LR = 0.009000, min batch loss = 2.514328, test MAE = 1.156975
step: 600, LR = 0.009000, min batch loss = 0.342869, test MAE = 0.620734
step: 700, LR = 0.009000, min batch loss = 1.031895, test MAE = 0.657341
step: 800, LR = 0.009000, min batch loss = 1.104359, test MAE = 0.712839
step: 900, LR = 0.009000, min batch loss = 2.461146, test MAE = 0.574431
step: 1000, LR = 0.008100, min batch loss = 0.230700, test MAE = 0.538429
step: 1100, LR = 0.008100, min batch loss = 0.020479, test MAE = 0.779294
step: 1200, LR = 0.008100, min batch loss = 0.424983, test MAE = 0.525701
step: 1300, LR = 0.008100, min batch 

classification

In [20]:
from __future__ import division

In [21]:
train_label.shape, test_label.shape

((70656, 8, 1), (17664, 8, 1))

In [22]:
train_label_l = train_label[:, -1, :]
test_label_l = test_label[:, -1, :]

In [23]:
train_label_l.reshape(1, -1).shape, test_label_l.shape

((1, 70656), (17664, 1))

In [24]:
from sklearn.preprocessing import Binarizer
pre = Binarizer(threshold = 1.01)
b_train_label = pre.transform(train_label_l.reshape(1, -1))
b_test_label = pre.transform(test_label_l.reshape(1, -1))

In [25]:
c_train_label = 1 - b_train_label[0]
c_test_label = 1 - b_test_label[0]

In [26]:
c_train_label.shape

(70656,)

In [27]:
# change from Indice to Vector
''''''
def makeIndicatorVars(T):
    # Make sure T is two-dimensiona. Should be nSamples x 1.
    if T.ndim == 1:
        T = T.reshape((-1,1))    
    return (T == np.unique(T)).astype(int)

In [28]:
v_train_label = makeIndicatorVars(c_train_label.reshape(-1, 1))
v_test_label = makeIndicatorVars(c_test_label.reshape(-1, 1))

In [43]:
batch_size = 128
patch_size = 7
n_hidden = 64
n_features1 = 70
n_features2 = 12
n_steps = 8
n_labels = 2
ratio = 0.2

total_size = train_old_label.shape[0]
print(total_size)

70656


In [44]:


graph = tf.Graph()
with graph.as_default():
    inputs1 = tf.placeholder("float32", [None, n_steps, n_features1])
    inputs2 = tf.placeholder("float32", [None, n_steps, n_features2])
    labels = tf.placeholder("float32", [None, n_labels])

    weights = {
        'layer1': tf.Variable(tf.truncated_normal([patch_size, 1, 1, 16], stddev=0.1)),
        'layer2': tf.Variable(tf.truncated_normal([patch_size, 1, 16, 16], stddev=0.1)),
        'layer3': tf.Variable(tf.truncated_normal([10 * 7 * 16, n_hidden], stddev=0.1)),
        'out': tf.Variable(tf.random_normal([n_hidden+12, n_labels])),
    }
    biases = {
        'layer1': tf.Variable(tf.zeros([16])),
        'layer2': tf.Variable(tf.constant(1.0, shape=[16])),
        'layer3': tf.Variable(tf.constant(1.0, shape=[n_hidden])),
        'out': tf.Variable(tf.random_normal([n_labels])),
    }
    
    def acc(predict, label):
        #correct_prediction = tf.equal(predicted_label, tf_train_label)
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(label, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        predict_event = tf.reduce_sum(tf.argmax(predict, 1))
        label_event = tf.reduce_sum(tf.argmax(label, 1))
        true_positive = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 2), tf.int64))
        true_negative = tf.reduce_sum(tf.cast(tf.equal((tf.argmax(predict, 1) + tf.argmax(label, 1)), 0), tf.int64))
        false_positive = predict_event - true_positive 
        false_negative = label_event - true_positive
        return accuracy, false_positive, false_negative, true_positive, true_negative
    def ROC(FP, FN, TP, TN):
        TP_percent = TP / (TP + FN) 
        FP_percent = FP / (FP + TN) 
        return TP_percent, FP_percent
    
    def PRC(FP, FN, TP, TN):
        precision = TP / (TP + FP + 1)
        recall = TP / (TP + FN + 1)
        f_score = TP / (TP + FN + FP)
        return precision, recall, f_score
    
    def cov_model(data, w, b):
        conv = tf.nn.conv2d(data, w['layer1'], [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b['layer1'])
        conv = tf.nn.conv2d(hidden, w['layer2'], [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + b['layer2'])
        #shape = hidden.get_shape().as_list()
        shape = tf.pack([tf.shape(hidden)[0], tf.shape(hidden)[1]*tf.shape(hidden)[2]*tf.shape(hidden)[3]])
        reshape = tf.reshape(hidden, shape)
        hidden = tf.nn.relu(tf.matmul(reshape, w['layer3']) + b['layer3'])
        return hidden
    
    def RNN(x, y, w, b):
        # Prepare data shape to match `rnn` function requirements
        # Current data input shape: (batch_size, n_steps, n_input)
        # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
        # Permuting batch_size and n_steps
        x = tf.transpose(x, [1, 0, 2])
        y = tf.transpose(y, [1, 0, 2])
        
        # Reshaping to (n_steps*batch_size, n_input)
        x = tf.reshape(x, [-1, n_features1])
        y = tf.reshape(y, [-1, n_features2])
        # Linear activation
        x = tf.reshape(x, [-1, 7, 10, 1])
        x = cov_model(x, w, b)
        
        concat = tf.concat(1, [x, y])
        # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
        list_data = tf.split(0, n_steps, concat)

        # Define a lstm cell with tensorflow
        lstm_cell = rnn_cell.BasicLSTMCell(n_hidden+12, forget_bias=1.0, state_is_tuple=True)

        # Get lstm cell output
        outputs, states = rnn.rnn(lstm_cell, list_data, dtype=tf.float32)

        # Linear activation, using rnn inner loop last output
        return tf.matmul(outputs[-1], w['out']) + b['out']
    
    pred = RNN(inputs1, inputs2, weights, biases)
    class_weight = tf.constant([ratio, 1.0 - ratio])
    weighted_pred = tf.mul(pred, class_weight) # shape [batch_size, 2]
    
    
    # Define loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(weighted_pred, labels)) # Softmax loss
    
    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.02
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
    
    
    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(labels,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    test_acc, FP, FN, TP, TN = acc(pred, labels)
    pre, rec, f_s = PRC(FP, FN, TP, TN)

In [45]:
st = time.time()
batch_size = 128
steps = 20000
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    test_feed_dict = {inputs1:test_data_n[:,:,:70], inputs2:test_data_n[:,:,70:], labels:v_test_label}
    for step in range(steps):
        off = step * batch_size % (total_size - batch_size)
        batch_data = train_data_n[off:off+batch_size, :, :]
        batch_label = v_train_label[off:off+batch_size, :]
        feed_dict = {inputs1:batch_data[:,:,:70], inputs2:batch_data[:,:,70:], labels:batch_label}
        l, _, lr = session.run([loss, op, learning_rate], feed_dict=feed_dict)
        if step % 100 == 0:
            print('Minibatch loss at step %d learning_rate = %f: loss = %f' % (step, lr, l))
            tp, fp, fn, tn, precision, recall, f_score= session.run([TP, FP, FN, TN, pre, rec, f_s], feed_dict=test_feed_dict)
            print("TP = %d, FP = %d, FN = %d, TN = %d" % (tp, fp, fn, tn))
            print("precision = %f, recall = %f, f_score = %f" % (precision, recall, f_score))
et= time.time()
print(et-st)

Initialized
Minibatch loss at step 0 learning_rate = 0.020000: loss = 5.600290
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.000000
Minibatch loss at step 100 learning_rate = 0.020000: loss = 0.005007
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.000000
Minibatch loss at step 200 learning_rate = 0.020000: loss = 0.086997
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.000000
Minibatch loss at step 300 learning_rate = 0.020000: loss = 0.011242
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.000000
Minibatch loss at step 400 learning_rate = 0.020000: loss = 0.006474
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.000000
Minibatch loss at step 500 learning_rate = 0.018000: loss = 0.003683
TP = 0, FP = 0, FN = 314, TN = 17350
precision = 0.000000, recall = 0.000000, f_score = 0.0